In [1]:
from torchvision.datasets import CIFAR10 
import numpy as np

#traindata = CIFAR10(root="cifar10_data")


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

ex = unpickle(f"cifar-10-python/cifar-10-batches-py/data_batch_1")

labels = []
data = [] 

for i in range(1,6):
    dic = unpickle(f"cifar-10-python/cifar-10-batches-py/data_batch_{i}")
    labels.extend(dic[b'labels'])
    data.extend(dic[b'data'])

/tmp/ipykernel_85098/2124082012.py:10: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  dict = pickle.load(fo, encoding='bytes')


In [2]:
data = np.array(data)
labels = np.array(labels)

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader
import matplotlib.pyplot as plt 

X = torch.tensor(data,dtype=torch.float32)/255.0
y = torch.tensor(labels,dtype=torch.long)

N = X.shape[0]
train_size = int(0.8 * N)
val_size = N - train_size
train_X,val_X = torch.split(X,[train_size,val_size])
train_y,val_y = torch.split(y,[train_size,val_size])

train_ds = TensorDataset(train_X,train_y)
val_ds = TensorDataset(val_X,val_y)

train_loader = DataLoader(train_ds,batch_size=128,shuffle=True)
val_loader = DataLoader(val_ds,batch_size=256,shuffle=False)

In [3]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# 2 conv layers with 1 dense layer 
class CNN(nn.Module):
    ## in_channels = if grayscale image -> 1 if RGB -> 3
    ## num_classes = number of final classes we want to predict
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()

        # 1 convolution layer: 1 input channel, 8 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)

        # Max pooling layer: 2x2 window, stride 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 2 convolution layer
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        
        # dense layer: if 28*28 pixels input image -> 2 max pool -> 16 (out channels) *  7*7 (2 max pool) 
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    ## x torch.tensor
    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        x = x.reshape(x.shape[0], -1)  # Flatten the tensor
        x = self.fc1(x)            # Apply fully connected layer
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"

input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 10  # digits 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 10  # Reduced for demonstration purposes




In [4]:
train_dataset = datasets.MNIST(root="dataset/", download=True, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


100%|██████████| 9.91M/9.91M [01:13<00:00, 136kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 372kB/s]
100%|██████████| 1.65M/1.65M [01:10<00:00, 23.4kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.93MB/s]


In [5]:
model = CNN(in_channels=1, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        # Move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass: compute the model output
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass: compute the gradients
        optimizer.zero_grad()
        loss.backward()

        # Optimization step: update the model parameters
        optimizer.step()



Epoch [1/10]


100%|██████████| 938/938 [00:13<00:00, 67.06it/s]


Epoch [2/10]


100%|██████████| 938/938 [00:14<00:00, 63.52it/s]


Epoch [3/10]


...

100%|██████████| 938/938 [00:14<00:00, 63.76it/s]


Epoch [9/10]


100%|██████████| 938/938 [00:14<00:00, 65.03it/s]


Epoch [10/10]


100%|██████████| 938/938 [00:14<00:00, 65.57it/s]


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")
    
    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Checking accuracy on training data
Got 59203/60000 with accuracy 98.67%
Checking accuracy on test data
Got 9826/10000 with accuracy 98.26%
